In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        w = torch.ones(6,1,5,5).cuda()
        x = F.conv2d(x,w)
        x = F.max_pool2d(F.relu(x), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

x = torch.ones(64,1,32,32).cuda();
net = Net()
for i in range(10000):
    net(x)
    print i;

In [ ]:
def ssim_loss(x):
    #x with shape [batch_size,num_features]
    #ssim_layer = ssim.SSIM(reshape = True,size_average=False,window_size = 11)
    delta = 0.01
    ssim_l = torch.Tensor(x.shape);
    num_feas = ssim_l.size(1)
    x_delta = x.clone();
    for i in range(num_feas):
        x_delta[:,i] = x_delta[:,i]+delta;
        
        ssim_l_i =torch.ones(64)
        #print ssim_l_i.mean()
        ssim_l[:,i] = ssim_l_i;
        
        x_delta[:,i] = x_delta[:,i]-delta;
        print i
    #with shape[batch_size,num_features]
    return ssim_l

x = torch.ones(64,32*32*3).cuda()
ssim_loss(x)

In [ ]:
import torch.nn as nn
import torch
DIM = 128
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        preprocess = nn.Sequential(
            nn.Linear(128, 4 * 4 * 4 * DIM),
            #nn.BatchNorm2d(4 * 4 * 4 * DIM),
            nn.ReLU(True),
        )


        self.preprocess = preprocess


    def forward(self, input):
        output = self.preprocess(input)
        return output
g_net = Generator()

a = torch.randn(128,128)
o = g_net(a)
print o.size()
print g_net

In [ ]:
import torch
import numpy as np
def cov_function(x):
    batch_size = x.size()[0]
    a = x - x.mean(dim=0,keepdim=True)
    cov_matrix = torch.matmul(a.t(),a)/(batch_size-1)
    return cov_matrix

gradients = torch.randn([10,20])
ma_distance = torch.mul(torch.matmul(gradients,cov_function(gradients)),gradients)
ma = ((torch.sqrt(torch.sum(ma_distance,dim=1)) - 1) ** 2).mean()
print ma_distance
print ma

print ((gradients.norm(2, dim=1) - 1) ** 2).mean()

In [ ]:
import torch
import numpy as np
def var_function(x):
    a = x - x.mean(dim=0,keepdim=True)
    #var_matrix = a.norm(2,dim=0)/5
    var_matrix = torch.sum(a ** 2,dim=0)/5
    return var_matrix

x = torch.rand(5,10)
x_numpy = x.numpy()

print np.var(x_numpy,axis=0)
varx = var_function(x)
# print x
print varx
# print x / var

#print x.mean(dim=0)

In [ ]:
import torch
from torch.autograd import Variable


a = torch.Tensor([[1,2]])
c = torch.ones(2,2)
b = torch.Tensor([[3,4]])
torch.mul(a,b)

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 23 14:02:48 2017

@author: lrh
"""


import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='/mnt/hgfs/ubuntu14/dataset', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/mnt/hgfs/ubuntu14/dataset', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


print len(trainloader)

# get some random training images
dataiter = iter(trainloader)

##FloatTensor  [batch_size,channel,height,width]
##[4,1,32,32]
images, labels = dataiter.next()


# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
plt.show()

In [ ]:
# -*- coding: utf-8 -*-

import torch
import torchvision
import torchvision.transforms as transforms

########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='/mnt/hgfs/ubuntu14/dataset', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/mnt/hgfs/ubuntu14/dataset', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

########################################################################
# Let us show some of the training images, for fun.

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))


########################################################################
# 2. Define a Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Copy the neural network from the Neural Networks section before and modify it to
# take 3-channel images (instead of 1-channel images as it was defined).

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

########################################################################
# 4. Train the network
# ^^^^^^^^^^^^^^^^^^^^
#
# This is when things start to get interesting.
# We simply have to loop over our data iterator, and feed the inputs to the
# network and optimize

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

########################################################################
# 5. Test the network on the test data
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#
# We have trained the network for 2 passes over the training dataset.
# But we need to check if the network has learnt anything at all.
#
# We will check this by predicting the class label that the neural network
# outputs, and checking it against the ground-truth. If the prediction is
# correct, we add the sample to the list of correct predictions.
#
# Okay, first step. Let us display an image from the test set to get familiar.

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

########################################################################
# Okay, now let us see what the neural network thinks these examples above are:

outputs = net(Variable(images))

########################################################################
# The outputs are energies for the 10 classes.
# Higher the energy for a class, the more the network
# thinks that the image is of the particular class.
# So, let's get the index of the highest energy:
_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

########################################################################
# The results seem pretty good.
#
# Let us look at how the network performs on the whole dataset.

correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

########################################################################
# That looks waaay better than chance, which is 10% accuracy (randomly picking
# a class out of 10 classes).
# Seems like the network learnt something.
#
# Hmmm, what are the classes that performed well, and the classes that did
# not perform well:

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


In [ ]:
#%%
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

"""
input:[batch_size,in_channel,height,width]
kernel:[out_channel,in_channel,kh,kw]
"""
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        #(28-5+1)/2=12
        self.conv2 = nn.Conv2d(6, 16, 5)
        #(12-5+1)/2=4
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        print "after conv1 size is {}".format(x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print "after conv2 size is {}".format(x.size())
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

print "hello world"
input = Variable(torch.Tensor(np.random.randint(1,10,size=(1,1,32,32))))
print net.forward(input)

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
print torch.Tensor([1,2])

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        
    def forward(self,x):
        x = self.conv1(x)
        return x
    
net= Net()
print net
input = Variable(torch.Tensor(1,1,32,32))
net(input)

In [ ]:
import torch
not torch.cuda.is_available()

In [ ]:
import torch
from torch.autograd import Variable
import torch.optim as optim
from torch.nn.parameter import Parameter
learning_rate = 0.009;
one = torch.FloatTensor([1]);

x = Parameter(torch.Tensor([10]),requires_grad=True);

print type(x)
def loss_fun(x):
    loss = x**2-10*x+50
    return loss

optimizer = optim.Adam([x], lr = 0.1)
#I think it's a much better design choice to keep Variables immutable,
#immutable : so it will rebuild a new variable.
########################--apaszke
for i in range(1000):
    l = loss_fun(x)
    loss = l**2
    
    #loss.backward()
    
    #print x.grad
    
    ####intermediary variable grad is None.!!!
    #print l.grad
    
    #x.grad.data.fill_(0)
    #x.data.sub_(learning_rate*x.grad.data)
    #print x
    #x.grad = None
    #x.grad.data.fill_(0)
    #print x
    #print "grad:{}".format(x.grad)
    #print type(x)
    #x = Variable((x - learning_rate*x.grad).data,requires_grad=True)
    
    optimizer.zero_grad()
    loss.backward(one)
    #print x.grad
    optimizer.step()
print x

In [ ]:
import torch
a = Variable(torch.Tensor([[1,2]]));
def f(a):
    a[0][1] = 5

f(a)
print a
    

In [ ]:
def F():
    for i in range(5, 9):
        for j in range(21, 29):
            x = yield i
            print x
            y = (yield j) * 100
            x += y
            print '>>>', x

gen = F()
gen.send(None)
#a2 = gen.next()
#gen.next()
a1 = gen.send(66)
a2 = gen.send(77)
print a1
print a2

In [ ]:
def consumer():
    r = -5
    for i in xrange(3):
        n = yield r
        print n
        
c = consumer()

print "first run generator:{}".format(c.next())

print "second run generator:{}".format(c.next())

print c.next()

#c.send(10)

In [ ]:
def f():
    i = 0
    for n in xrange(3):
        i = yield i
        print "i in function:{}".format(i)
        
c = f()
print c.next()

c.send(10)
c.send(20)
c.send(30)

In [ ]:
prefix = ''
'.' if prefix else "hello world"

In [ ]:
from __future__ import print_function
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
print(torch.Tensor([1,2]))


############################################module
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.weight = nn.Parameter(torch.Tensor([[1,2]]))
        self.bias = nn.Parameter(torch.Tensor([[1]]))
        
    def forward(self,x):
        x = nn.Sigmoid()(self.weight.mm(x)+self.bias)
        return x
    
net= Net()
print(net)
#input = Variable(torch.Tensor(1,1,32,32))

input = Variable(torch.Tensor([[1],[2]]))

#for parameter in net.parameters():
#    print parameter

labels = 1;

##############################################loss
def loss_fn(labels,logits):
    l = labels*torch.log(logits)+(1-labels)*(torch.log(1-logits))
    return -l;

def calc_gradient_penalty(nets,input):
    
    output = nets(input)
    
    #output gradient w.r.t input
    gradients = autograd.grad(outputs=output,inputs=input,grad_output=torch.ones(output.size()).cuda()...
                             ,create_graph=True);
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty


###############################################optimizer
optimizer = optim.SGD(net.parameters(),lr = 0.01, momentum=0.9)



############################################training
running_loss = 0.0
for i in xrange(1000):
    logits = net(input)
    loss = loss_fn(labels,logits)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%100 == 0:
        print ("loss is %3f" %(loss.data[0][0]))
        
print(net(input).data[0][0])

In [ ]:
import torchvision.utils as vutils

vutils.save_image(tensor, filename, nrow=8, padding=2, normalize=False, range=None, scale_each=False)


In [ ]:
import torch
a = torch.Tensor([[1,2]])
b = torch.Tensor([[1],[2]])
print a.mm(b)

In [ ]:
import torch
from torch.nn.parameter import Parameter
from torch.autograd import Variable
w1 = Parameter(torch.Tensor([1]))
w2 = Parameter(torch.Tensor([2]))

y = w1*2
z = w2*y
out = z - 5 
out.backward()

print w2.grad

In [ ]:
# -*- coding: utf-8 -*-

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='/home/lrh/dataset/cifar-10', train=True,
                                        download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/home/lrh/dataset/cifar-10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')




for data in trainloader:
    # get the inputs
    inputs, labels = data
    # wrap them in Variable
    #inputs, labels = Variable(inputs), Variable(labels)
    #print "epoch:{}".format(epoch)
    #print "hello world"
    #inputs.detach()
    #vutils.save_image(tensor, filename, nrow=8, padding=2, normalize=False, range=None, scale_each=False)
    #vutils.save_image(inputs.detach(),"/home/lrh/grad.png")
    print inputs.dtype
    print "hello world"
    break;
# print len(trainset)
# print len(trainloader)      
# print('Finished Training')


In [ ]:
import torch 
a = torch.randn([5,6])
print a
a.detach()

In [ ]:
import torch
import numpy
from torch.autograd import Variable

a = torch.Tensor([1,2,3,4,5])
v_a = Variable(a)

a[1:2].ndim

In [ ]:
import numpy as np
def dense_to_one_hot(labels,class_num):
    height = labels.shape[0]
    a = np.zeros(shape=[height,class_num])
    a[np.arange(height),labels.squeeze()] = 1
    return a
    
labels = np.array([[1],[2],[3]])
labels = dense_to_one_hot(labels,10)
print labels





In [ ]:
import torch

a = torch.Tensor([[2,1,1],[3,4,3]])
print a
a,pos = a.max(dim=1)
print a
print pos

In [ ]:
import torch
def squash(x,dim):
    #we should do spuash in each capsule.
    #we use dim to select
    sum_sq = torch.sum(x**2,dim=dim,keepdim=True)
    print sum_sq
    sum_sqrt = torch.sqrt(sum_sq)
    return (sum_sq/(1.0+sum_sq))* x/sum_sqrt
    

a = torch.Tensor([[1,20,20],[4,5,6]])
b = squash(a,dim=1)
print b
print torch.mean(b**2,dim=1)

In [ ]:
import torch

x = torch.zeros_like([4,1152,8])
x = torch.stack([x] * 10, dim=2).unsqueeze(3)
print x.shape

zero = torch.zeros([1]).double()
print zero

In [ ]:
import torch
x = torch.Tensor([[4,2],[-4,-2]])
zero = torch.zeros([1])
x.cuda()
print torch.max(zero,x)**2


In [ ]:
import numpy as np
def dense_to_one_hot(labels,class_num):
    
    height = labels.shape[0]
    a = np.zeros(shape=[height,class_num])
    a[np.arange(height),labels] = 1
    return a

labels = np.array([5,6,7])
labels = dense_to_one_hot(labels,10)
print labels

In [ ]:
import torch
torch.cuda.is_available()

In [2]:
import torch.nn as nn
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
class classify_net(nn.Module):

    def __init__(self):
        super(classify_net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5,padding=(2,2))
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5,padding=(2,2))
        self.bn2 = nn.BatchNorm2d(16)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(F.relu(x))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(F.relu(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x,dim=1)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = classify_net().cuda()
pkl_dir = "/home/lrh/program/git/pytorch-example/mnist_cnn/pkls/mnist_init_300epoch.pkl"
net.load_state_dict(torch.load(pkl_dir))


net.zero_grad()

data = dset.MNIST(root="/home/lrh/dataset/mnist",train = False,download=True,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]))
dataloader = torch.utils.data.DataLoader(data,batch_size=64,shuffle=True,drop_last=True)

for i,data in enumerate(dataloader,0):
    images,labels = data
    images = images.cuda()
    labels = labels.cuda()
    pred = net(images)
    _,logits = torch.max(pred,1)
    print pred
    print labels
    print logits
    break


tensor([[ 2.9648e-05,  1.4163e-03,  5.4473e-05,  2.8602e-04,  2.1613e-02,
          5.4275e-04,  8.7522e-06,  2.2030e-02,  1.1516e-02,  9.4250e-01],
        [ 1.3895e-05,  9.9835e-01,  1.7850e-04,  5.2711e-05,  8.3725e-04,
          3.1201e-06,  1.8867e-04,  2.2544e-04,  1.3708e-04,  8.6281e-06],
        [ 3.2763e-03,  1.7253e-03,  6.1538e-01,  4.9966e-03,  9.7277e-05,
          2.8367e-05,  2.1504e-05,  3.5531e-01,  9.5612e-03,  9.6103e-03],
        [ 6.5695e-05,  4.2320e-07,  8.1510e-05,  1.1792e-05,  1.0927e-03,
          8.4285e-06,  2.9287e-07,  2.8563e-02,  7.1859e-05,  9.7010e-01],
        [ 1.2230e-05,  1.1033e-06,  2.8461e-03,  9.9654e-01,  1.6540e-11,
          9.0017e-05,  4.7520e-10,  4.3122e-04,  7.4502e-05,  6.8967e-06],
        [ 9.9589e-01,  4.4169e-07,  2.1042e-03,  3.0515e-04,  7.9895e-08,
          7.0174e-04,  2.2959e-04,  6.1019e-04,  1.3459e-04,  2.8328e-05],
        [ 1.0339e-06,  2.5104e-06,  1.5158e-03,  9.6219e-01,  6.0184e-07,
          3.0654e-02,  8.1808e-0